<a href="https://colab.research.google.com/github/plttraining/hsf_matplotlib_notebooks/blob/main/ep07_dimuonspectrum_awkward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **If on Colab**

Please click on the bage and then run the code below

In [ ]:
! git clone https://github.com/plttraining/hsf_matplotlib_notebooks.git
! cd hsf_matplotlib_notebooks/; pip install -r requirements.txt ; bash postBuild
% cd hsf_matplotlib_notebooks/



You might see a warning to Restart the Runtime. This is expected. Just go to the `Kernel` tab and click on `Restart runtime`. Without this restart on Colab you might be able not use mplhep properly.

You only have to do this once per notebook on Google Colab.


# Looking at the dimuon spectrum over a wide energy range

<h3>Learning goals</h3>
<ul>
    <li>Relativistic kinematics.
    <li>Mesons.
</ul>

<b>Background</b>

To determine the mass ($m$) of a particle you need to know the 4-momenta of the particles ($\mathbf{P}$) that are detected after the collision: the energy ($E$), the momentum in the x direction ($p_x$), the momentum in the y direction ($p_y$), the momentum in the z direction ($p_z$).

$$\mathbf{P} = (E,p_x,p_y,p_z)$$


\begin{equation*} m = \sqrt{E^2-(p_x^2+p_y^2 + p_z^2)} \end{equation*}

Some particles are very unstable and decay (turn into) to two or more other particles. In fact, they can decay so quickly, that they never interact with your detector! Yikes!

However, we can reconstruct the parent particle (sometimes referred to as <b>the initial state particle</b>) and its 4-momentum by adding the 4-momenta of the child particles (sometimes referred to as <b>the decay products</b>). 

$$\mathbf{P_{\rm parent}} = \mathbf{P_{\rm child 0}} + \mathbf{P_{\rm child 1}} + \mathbf{P_{\rm child 2}} + ...$$



which breaks down into...

$$E_{\rm parent} = E_{\rm child 0} + E_{\rm child 1} + E_{\rm child 2} + ...$$

$$p_{\rm x parent} = p_{\rm x child 0} + p_{\rm x child 1} + p_{\rm x child 2} + ...$$

$$p_{\rm y parent} = p_{\rm y child 0} + p_{\rm y child 1} + p_{\rm y child 2} + ...$$

$$p_{\rm z parent} = p_{\rm z child 0} + p_{\rm y child 1} + p_{\rm z child 2} + ...$$


<b>Let's code!</b>

Here is some very, very basic starter code. It reads in data from the CMS experiment. 
 


<h2><font color="red">Challenge!</font></h2>

Use the sample code to find the mass of the particle that the two muons came from (parent particle). 

To do this, you will need to loop over all pairs of muons for each collision, sum their 4-momenta (energy, px, py, and pz) and then use that to calculate the invariant mass. 

Do this for all pairs of muons for the case where the muons have opposite charges.


<i>Hint!</i>

It is very likely that a particle exists where there is a peak in the data. However, this is not always true. 
A peak in the data is most likely the mass of a particle. You can look at the approximate mass to figure out which particle 
is found in the data.

Your histogram should look something like the following sketch. The value of the peaks should be the mass of a particle. You should be able to find two particles in their ground state. <a href="http://en.wikipedia.org/wiki/J/psi_meson">Check your answer for the first particle!</a> <a href="http://en.wikipedia.org/wiki/Upsilon_meson">Check your answer for the second particle!</a> 

In [ ]:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/particle-physics-playground/playground/master/activities/images/dimuons_sketch.jpeg')

In [ ]:
import numpy as np
import h5py
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep

Decide which styling you want to use

In [ ]:
# plt.style.use("default") # This is the default style for matplotlib # Do not change this cell if you desire this option
# hep.style.use("ROOT") # This is the mplhep style # Uncomment this line for this styling.

In [ ]:
event=h5py.File('./data-ep07-dimuonspectrum/dimuon100k.hdf5',mode='r') #  make sure you have the correct path to the dimuon file

In [ ]:
e=event['muons/e'][:]
px =event['muons/px'][:]
py =event['muons/py'][:]
pz =event['muons/pz'][:]
q = event['muons/q'][:]

# We will check for muons that do not pass the kinematics
print(len(px)) # number of muons

# see if there are any anomalies and clean them out
cut=(e**2 - (px**2 + py**2 + pz**2)) <0

print(sum(cut)) #count how many anomalies

We can use numpy to clean our arrays from anomalous events

In [ ]:
e=np.delete(e,cut)
px,py,pz=np.delete(px,cut),np.delete(py,cut),np.delete(pz,cut)
q=np.delete(q,cut)

In [ ]:
# let's calculate the mass
M=(e**2 - (px**2 + py**2 + pz**2))**.5

In [ ]:
# Make a histogram of the values of the Mass

### Your code Here ###



######################

plt.xlabel('$\mu_{mass}$ [GeV]')
plt.title('Muon Mass spectrum')
plt.show()

## Excercise :
Using our knowledge so far, zoom in to fix the above plot to **visually** estimate the mass of the muon

In [ ]:
#### Your code here ####




#### 
# Where is the value of the peak at?
####

# Let's make the dimuon spectrum

We need to calculate the sum the energies at the event level 


**REMEMBER** 
$$E_{\rm parent} = E_{\rm child 0} + E_{\rm child 1} + E_{\rm child 2} + ...$$

$$p_{\rm x parent} = p_{\rm x child 0} + p_{\rm x child 1} + p_{\rm x child 2} + ...$$

$$p_{\rm y parent} = p_{\rm y child 0} + p_{\rm y child 1} + p_{\rm y child 2} + ...$$

$$p_{\rm z parent} = p_{\rm z child 0} + p_{\rm y child 1} + p_{\rm z child 2} + ...$$

Let's assume that each event only has 2 muons.
We will use the awkward arrays function `unflatten` because we can make pairs of muons this way.

In [ ]:
e=ak.unflatten(e,2)
px=ak.unflatten(px,2)
py=ak.unflatten(py,2)
pz=ak.unflatten(pz,2)
q= ak.unflatten(q,2)

First we have to make a cut. We use `ak.sum` to add the charges of the muon pairs.
If the charges are opposite the sum is 0. We use this information to select our muon pairs.

Here we define a cut to keep only those pairs of muons

In [ ]:
cut = (ak.sum(q,axis=-1) == 0) # This returns a boolean array for each event. If it contains opposite charged muons is shall be True
print(cut)

Now lets keep only those pairs of muons that have opposite charge. Then we will sum the 4-momenta components and calculate the event mass.

In [ ]:
# Keep only the muons with opposite charges
e=e[cut]
px,py,pz=px[cut],py[cut],pz[cut]
q=q[cut]

In [ ]:
# we use axis = 1 because it squishes the row dimension and gives us 1 value per row (i.e. 1 value per event)
e = np.sum(e,axis=1) 
px = np.sum(px,axis=1)
py = np.sum(py,axis=1)
pz = np.sum(pz,axis=1)
q = np.sum(q,axis=1) # this should be an array of zeroes. Check!!!

Now calculate the mass per event

In [ ]:
Mass=np.sqrt((e**2-(px**2+py**2+pz**2)))

In [ ]:
##### Your code here #####
# You should make a histogram that looks like the one at the end of this notebook. 
# It won't be a perfect match so don't worry to much if it seems odd.




##########################


plt.xlabel('mass (GeV/$c^2$)')
plt.ylabel('Events')
plt.yscale('symlog')
plt.xscale('symlog')
plt.title('Mass of dimuons per event')

plt.show()


Depending on what you did, you may see hints of particles below 20 GeV/c$^2$. It is possible you see signs of other particles at even higher energies. Plot your masses over a wide range of values, but then zoom in (change the plotting range) on different mass ranges to see if you can identify these particles.  

In [ ]:
Image(url='https://twiki.cern.ch/twiki/pub/CMSPublic/HLTDiMuon2017and2018/CMS_HLT_DimuonMass_Inclusive_2017.png')